In [27]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.svm import SVR
from sklearn.linear_model import SGDRegressor
from sklearn.decomposition import PCA
from sklearn.cross_decomposition import PLSRegression
from sklearn.metrics import mean_squared_error
import os

In [28]:
exps_dir = "../../exps"
if os.path.exists(exps_dir) == False: # tạo thư mục (nếu chưa có)
  os.makedirs(exps_dir, exist_ok=True)

save_dir = f"{exps_dir}"
os.makedirs(save_dir, exist_ok=True)
test_size=0.33
seed=42

In [29]:
data= pd.read_excel('../../data/No Finance Banking.xls')

In [30]:
data['ResearchDevFund_ratio']=data.ResearchDevFund/data.TotalRevenue
data['SciTechFund_ratio']=data.SciTechFund/data.TotalRevenue
# Tính ROA
data['ROA'] = data['EBIT'] / data['TOTALASSETS']

# Tính ROE
data['ROE'] = data['EBIT'] / data['OWNERSEQUITY']
# data['Intangible Assets']=data['CostIntanFA']/data['TOTALASSETS']
# data['Intangible Assets1']=data['AccDepIntanFA']/data['TOTALASSETS']
# data['IAoverTA']
data['Firmsize_w']=np.log(data['TOTALASSETS'])
# data['T.Debt']=data['TotalDebtoverOE']/data['TOTALASSETS']
# data['T_Debt1']=data['TotalDebtoverTA']/data['TOTALASSETS']
data

c:\Users\PC\miniconda3\envs\uii_prj\lib\site-packages\pandas\core\arraylike.py:402: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


,id,Company,Index,Year,CashRatio,EBIT,EBITDA,GrossProfitMargin,EBITDARatio,EBITRatio,...,ReceivedInterest,NetCashChange,CashandEquiBeg,ExcRateFluctuation,CashandEquiEnd,ResearchDevFund_ratio,SciTechFund_ratio,ROA,ROE,Firmsize_w
0,A32,Công ty 32,UPCoM,2006,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,A32,Công ty 32,UPCoM,2007,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,A32,Công ty 32,UPCoM,2008,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,A32,Công ty 32,UPCoM,2009,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,A32,Công ty 32,UPCoM,2010,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26624,YTC,XNK Y tế TP.HCM,UPCoM,2018,0.08,3.617160e+10,4.021374e+10,0.1744,0.0760,0.0683,...,0.0,-4.586848e+10,1.017774e+11,0.0,5.590891e+10,0.038055,0.0,0.047042,0.640009,27.368264
26625,YTC,XNK Y tế TP.HCM,UPCoM,2019,0.21,3.319102e+10,3.700330e+10,0.1836,0.0726,0.0651,...,0.0,8.299045e+10,5.590891e+10,-14871234.0,1.388845e+11,0.043871,0.0,0.044125,0.554094,27.346265
26626,YTC,XNK Y tế TP.HCM,UPCoM,2020,0.03,-1.965803e+10,-1.455882e+10,0.0861,-0.0257,-0.0347,...,0.0,-1.140651e+11,1.388845e+11,0.0,2.481934e+10,0.037316,0.0,-0.019746,-1.460577,27.626546
26627,YTC,XNK Y tế TP.HCM,UPCoM,2021,0.02,9.693024e+08,5.319168e+09,0.0868,0.0088,0.0016,...,0.0,-6.547163e+09,2.481934e+10,0.0,1.827217e+10,0.000000,0.0,0.001241,0.044573,27.383589


In [59]:
df.describe()

,Year,ROE,ROA,CashRatio,ResearchDevFund,IAoverTA,TotalDebtoverTA,Firmsize_w,ResearchDevFund_ratio,SciTechFund_ratio
count,15654.000000,15654.000000,15654.000000,15654.000000,1.565400e+04,15654.000000,15654.000000,15654.000000,1.565400e+04,1.565400e+04
mean,2015.401750,0.145644,0.061214,0.651027,6.464723e+10,0.002949,0.250818,26.741617,4.467571e+07,9.535257e+05
std,4.281809,2.982552,0.275499,3.391730,4.712675e+11,0.021450,0.627543,1.498340,3.818193e+08,1.931119e+07
min,2006.000000,-258.063005,-28.849256,0.000000,-5.663668e+09,0.000000,0.000000,16.721515,-8.090954e+06,-4.949245e-03
25%,2012.000000,0.048992,0.019456,0.060000,1.286228e+09,0.000000,0.020000,25.736190,7.275095e-03,0.000000e+00
50%,2016.000000,0.143269,0.056547,0.190000,7.838702e+09,0.000000,0.180000,26.628220,6.286977e-02,0.000000e+00
75%,2019.000000,0.254024,0.104903,0.540000,2.763202e+10,0.000000,0.360000,27.631648,6.285002e+06,0.000000e+00
max,2022.000000,144.410746,1.227954,235.600000,2.106306e+13,0.790000,27.100000,33.071343,2.302689e+10,1.153665e+09


In [31]:
for i in data.columns:
    print(i,end=',')

id,Company,Index,Year,CashRatio,EBIT,EBITDA,GrossProfitMargin,EBITDARatio,EBITRatio,QuickRatio,CurrentRatio,LongDetboverOE,LongDebtoverTA,TotalDebtoverOE,TotalDebtoverTA,ShortDebtoverOE,ShortDebtoverTA,IAoverTA,TAoverOE,EBITDAoverShortDebtandExpenses,EBTMargin,NetProfitMarginRatio,TotalAssetTurnover,TotalEquityRatio,Tỷsuấtlợinhuậntừkinhdoanh,LNSTCFODoanhthu,Giátrịcổphiếu,TotalRevenue,SalesDeductions,NetRevenue,COGS,GrossProfit,RevenueFinancialActivities,FinancialExpenses,InterestExpenses,Lãilỗtừcôngtyliêndoanh,SellingExpenses,AdminExpensesCons,OperatingIncomeCons,OtherRevenue,OtherExpenses,OtherIncome,AccountingIncomePreTax,CorpTaxExpenses,CurrentCorpTax,DeferredIncomeTax,IncomeAfterDefTax,MinorityInterest,Cổđôngcủacôngtymẹ,EarningPerShare,DilutedEarningsPerShare,CURRENTASSETS,Cash,CashEquivalents,ShortTermInvestments,TradingSecurities,AllowanceDeclineInSec,HTMInvestments,CurrentReceivables,CurTradeReceivables,CurAdvancestoSuppliers,RelatedPartiesReceivables,ConstructionReceivables,Sho

In [32]:
'id,Company,Index,Year,CashRatio,EBIT,EBITDA,GrossProfitMargin,EBITDARatio,EBITRatio,QuickRatio,CurrentRatio,LongDetboverOE,LongDebtoverTA,TotalDebtoverOE,TotalDebtoverTA,ShortDebtoverOE,ShortDebtoverTA,IAoverTA,TAoverOE,EBITDAoverShortDebtandExpenses,EBTMargin,NetProfitMarginRatio,TotalAssetTurnover,TotalEquityRatio,Tỷsuấtlợinhuậntừkinhdoanh,LNSTCFODoanhthu,Giátrịcổphiếu,TotalRevenue,SalesDeductions,NetRevenue,COGS,GrossProfit,RevenueFinancialActivities,FinancialExpenses,InterestExpenses,Lãilỗtừcôngtyliêndoanh,SellingExpenses,AdminExpensesCons,OperatingIncomeCons,OtherRevenue,OtherExpenses,OtherIncome,AccountingIncomePreTax,CorpTaxExpenses,CurrentCorpTax,DeferredIncomeTax,IncomeAfterDefTax,MinorityInterest,Cổđôngcủacôngtymẹ,EarningPerShare,DilutedEarningsPerShare,CURRENTASSETS,Cash,CashEquivalents,ShortTermInvestments,TradingSecurities,AllowanceDeclineInSec,HTMInvestments,CurrentReceivables,CurTradeReceivables,CurAdvancestoSuppliers,RelatedPartiesReceivables,ConstructionReceivables,ShortLoanReceivables,OtherShortLoanRecei,ShortBadDebtProvision,AssetShortageAwait,Invetories,ObsoleteInventories,TotalOtherCA,ShortPrepaidExpenses,VATDeductions,TaxandStatutory,TradinGOVBondsAs,OtherCA,TOTALNONCURASSETS,Longreceivables,LongTradeReceivables,LongAdvancesSuppliers,WorkingCapSubUnits,LongRelatedPartiesRecei,LongLoanReceivables,OtherLongRecei,LongBadDebtProvision,FixedAssets,ResidualValTangFA,CostTangibleFA,AccDepreTanFA,ResidualFiLeasedFA,CostLeasedFA,AccDepLeaseFA,ResiduIntangFA,CostIntanFA,AccDepIntanFA,NetPropInvestment,CostPropInvest,AccuDepProInv,LongWIPAsset,LongWIPCost,LongConstWIP,TotalLongInvestments,InvestSubsidiary,InvestJoint,OtherLongInvest,LongProvisionInvest,HTMInvest,ToOtherLongAssets,LongPrepaidExp,DefTaxAssets,LongEqpMat,OtherLongAssets,Goodwill,TOTALASSETS,LIABILITIES,CurrentLiab,CurTradePayables,CurAdvancesCustomers,StatuObligations,EmployPayables,CurAccruExp,CurPayParties,PayConstContract,ShortDeferedReve,OtherCurPay,ShortLoanPayFinance,ShortProviPay,WelfareFund,StabliFund,TradingGOVBondsLiab,NonCurLiab,LongTradePay,LongAdvanceCustomers,LongAccruedExp,WorkingCapSubunits,LongPayParties,LongDeferredReve,OtherLongLiab,LongLoanDebts,ConvertibleBonds,PrefSharesNonCurLiab,DefTaxLiab,LongProviBadDebts,SciTechFund,OWNERSEQUITY,Vốnvàcácquỹ,CharteredCap,CommonShares,PrefShares,SharePremium,Quyềnchọnchuyểnđổitráiphiếu,OtherEquities,TreasuryShares,AssetRevaluation,ForeignExDiff,ResearchDevFund,EntReorgAssis,FinResFund,OtherOEFunds,UndistributedEarns,PrevUndisEarns,CurUndisEarns,ConstructInvestFund,NonControlInterest,ToOtherOEFund,Nguồnkinhphí,FAfromOtherFund,TOTALLIABANDOE,CFfromOperating,TiềnthutừbánhàngCungcấ,Tiềnchitrảchongườicungc,Tiềnchitrảchongườilaođộ,CFOIntExp,Tiềnchinộpthuếthunhậpdo,Tiềnthutừhoạtđộngkinhdo,Tiềnchikhácchohoạtđộngk,CFfromInvesting,PurchaseandConstrucFA,DisposalofAssets,LoantoRelatedParties,CollectedLoantoParties,EquityInvestinParties,CollectionInvestEntities,IntDivReceived,CFfromFinancing,IssuanceOrdShares,CapReturnedtoOwners,BondIssuance,LoanPMT,PmttoFinLeaser,PaidDivid,ReceivedInterest,NetCashChange,CashandEquiBeg,ExcRateFluctuation,CashandEquiEnd,ResearchDevFund_ratio,SciTechFund_ratio,ROA,ROE,Intangible Assets,Intangible Assets1,T.Debt,T.Debt1,'

'id,Company,Index,Year,CashRatio,EBIT,EBITDA,GrossProfitMargin,EBITDARatio,EBITRatio,QuickRatio,CurrentRatio,LongDetboverOE,LongDebtoverTA,TotalDebtoverOE,TotalDebtoverTA,ShortDebtoverOE,ShortDebtoverTA,IAoverTA,TAoverOE,EBITDAoverShortDebtandExpenses,EBTMargin,NetProfitMarginRatio,TotalAssetTurnover,TotalEquityRatio,Tỷsuấtlợinhuậntừkinhdoanh,LNSTCFODoanhthu,Giátrịcổphiếu,TotalRevenue,SalesDeductions,NetRevenue,COGS,GrossProfit,RevenueFinancialActivities,FinancialExpenses,InterestExpenses,Lãilỗtừcôngtyliêndoanh,SellingExpenses,AdminExpensesCons,OperatingIncomeCons,OtherRevenue,OtherExpenses,OtherIncome,AccountingIncomePreTax,CorpTaxExpenses,CurrentCorpTax,DeferredIncomeTax,IncomeAfterDefTax,MinorityInterest,Cổđôngcủacôngtymẹ,EarningPerShare,DilutedEarningsPerShare,CURRENTASSETS,Cash,CashEquivalents,ShortTermInvestments,TradingSecurities,AllowanceDeclineInSec,HTMInvestments,CurrentReceivables,CurTradeReceivables,CurAdvancestoSuppliers,RelatedPartiesReceivables,ConstructionReceivables,Sh

In [33]:
data.drop_duplicates(inplace=True)

In [34]:
len(data)

26622

In [35]:
# Thay thế giá trị inf bằng NaN
data.replace([np.inf, -np.inf], np.nan, inplace=True)

# Loại bỏ các dòng chứa NaN
# data.dropna(axis=0, how='any', inplace=True)

In [36]:
len(data)

26622

In [37]:
change_category = list((data.select_dtypes('object')).columns)
change_category

['id', 'Company', 'Index']

In [38]:
data[change_category]=data[change_category].astype('category')
data.dtypes

id                       category
Company                  category
Index                    category
Year                        int64
CashRatio                 float64
                           ...   
ResearchDevFund_ratio     float64
SciTechFund_ratio         float64
ROA                       float64
ROE                       float64
Firmsize_w                float64
Length: 201, dtype: object

In [39]:
category_columns=list((data.select_dtypes('category')).columns)
numeric_columns=list((data.select_dtypes('number')).columns)

In [40]:
df=data[['id','Company','Index','Year','ROE','ROA','CashRatio','ResearchDevFund','IAoverTA','TotalDebtoverTA','Firmsize_w','ResearchDevFund_ratio','SciTechFund_ratio']]

In [41]:
df

,id,Company,Index,Year,ROE,ROA,CashRatio,ResearchDevFund,IAoverTA,TotalDebtoverTA,Firmsize_w,ResearchDevFund_ratio,SciTechFund_ratio
0,A32,Công ty 32,UPCoM,2006,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,A32,Công ty 32,UPCoM,2007,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,A32,Công ty 32,UPCoM,2008,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,A32,Công ty 32,UPCoM,2009,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,A32,Công ty 32,UPCoM,2010,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
26624,YTC,XNK Y tế TP.HCM,UPCoM,2018,0.640009,0.047042,0.08,2.049601e+10,0.00,0.45,27.368264,0.038055,0.0
26625,YTC,XNK Y tế TP.HCM,UPCoM,2019,0.554094,0.044125,0.21,2.276410e+10,0.00,0.37,27.346265,0.043871,0.0
26626,YTC,XNK Y tế TP.HCM,UPCoM,2020,-1.460577,-0.019746,0.03,2.276410e+10,0.00,0.23,27.626546,0.037316,0.0
26627,YTC,XNK Y tế TP.HCM,UPCoM,2021,0.044573,0.001241,0.02,0.000000e+00,0.01,0.28,27.383589,0.000000,0.0


In [42]:
len(df)

26622

In [43]:
df.dropna(inplace=True)
len(df)

C:\Users\PC\AppData\Local\Temp\ipykernel_2952\2888176961.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.dropna(inplace=True)


15654

In [44]:
category_columns=list((df.select_dtypes('category')).columns)
numeric_columns=list((df.select_dtypes('number')).columns)
category_columns.append('Year')
numeric_columns.remove('Year')
print(category_columns)
print(numeric_columns)

['id', 'Company', 'Index', 'Year']
['ROE', 'ROA', 'CashRatio', 'ResearchDevFund', 'IAoverTA', 'TotalDebtoverTA', 'Firmsize_w', 'ResearchDevFund_ratio', 'SciTechFund_ratio']


In [45]:
# from sklearn.preprocessing import LabelEncoder,MinMaxScaler,OneHotEncoder,StandardScaler

# label_encoders = {}
# for column in category_columns:
#     label_encoder = LabelEncoder()
#     df[column] = label_encoder.fit_transform(df[column])
#     label_encoders[column] = label_encoder  # Lưu trữ label encoder nếu cần sau này

# scaler = StandardScaler()
# df[numeric_columns] = scaler.fit_transform(df[numeric_columns])
# df

In [61]:
data.info

<bound method DataFrame.info of         id          Company  Index  Year  CashRatio          EBIT  \
0      A32       Công ty 32  UPCoM  2006        NaN           NaN   
1      A32       Công ty 32  UPCoM  2007        NaN           NaN   
2      A32       Công ty 32  UPCoM  2008        NaN           NaN   
3      A32       Công ty 32  UPCoM  2009        NaN           NaN   
4      A32       Công ty 32  UPCoM  2010        NaN           NaN   
...    ...              ...    ...   ...        ...           ...   
26624  YTC  XNK Y tế TP.HCM  UPCoM  2018       0.08  3.617160e+10   
26625  YTC  XNK Y tế TP.HCM  UPCoM  2019       0.21  3.319102e+10   
26626  YTC  XNK Y tế TP.HCM  UPCoM  2020       0.03 -1.965803e+10   
26627  YTC  XNK Y tế TP.HCM  UPCoM  2021       0.02  9.693024e+08   
26628  YTC  XNK Y tế TP.HCM  UPCoM  2022       0.03  4.674561e+09   

             EBITDA  GrossProfitMargin  EBITDARatio  EBITRatio  ...  \
0               NaN                NaN          NaN        NaN  ... 

In [46]:
X = df.drop(columns=['ROE'])
y = df['ROE']
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# print(X_train.shape,X_test.shape)

In [47]:
# from sklearn.feature_selection import RFE
# from sklearn.linear_model import LinearRegression
# import pandas as pd

# # # Giả sử df là DataFrame của bạn, chứa cả biến độc lập và biến mục tiêu
# # # Xác định X là ma trận đặc trưng và y là vector biến mục tiêu
# # X = df.drop(columns=['target_column'])
# # y = df['target_column']

# # Sử dụng mô hình hồi quy tuyến tính
# model = LinearRegression()

# # Chọn số lượng đặc trưng tối ưu bằng cách chạy RFE
# # Ở đây, chúng ta chọn số lượng đặc trưng tối ưu là 5, bạn có thể thay đổi giá trị này tùy theo yêu cầu
# rfe = RFE(model, n_features_to_select=5)

# # Fit RFE trên dữ liệu
# rfe.fit(X, y)

# # In ra các đặc trưng được chọn và hạng của chúng
# selected_features = pd.DataFrame({'Feature': X.columns, 'Selected': rfe.support_, 'Rank': rfe.ranking_})
# print(selected_features)


In [48]:
df

,id,Company,Index,Year,ROE,ROA,CashRatio,ResearchDevFund,IAoverTA,TotalDebtoverTA,Firmsize_w,ResearchDevFund_ratio,SciTechFund_ratio
10,A32,Công ty 32,UPCoM,2016,0.172116,0.060481,0.47,5.585241e+10,0.00,0.22,26.865314,0.088759,0.0
11,A32,Công ty 32,UPCoM,2017,0.164296,0.057810,0.45,6.856909e+10,0.00,0.26,26.939857,0.112057,0.0
12,A32,Công ty 32,UPCoM,2018,0.189551,0.081188,0.22,8.093215e+10,0.00,0.01,26.873530,0.125240,0.0
13,A32,Công ty 32,UPCoM,2019,0.206177,0.106003,0.29,9.598561e+10,0.00,0.02,26.798452,0.138811,0.0
14,A32,Công ty 32,UPCoM,2020,0.174019,0.086321,0.18,1.124009e+11,0.00,0.00,26.914186,0.154273,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
26624,YTC,XNK Y tế TP.HCM,UPCoM,2018,0.640009,0.047042,0.08,2.049601e+10,0.00,0.45,27.368264,0.038055,0.0
26625,YTC,XNK Y tế TP.HCM,UPCoM,2019,0.554094,0.044125,0.21,2.276410e+10,0.00,0.37,27.346265,0.043871,0.0
26626,YTC,XNK Y tế TP.HCM,UPCoM,2020,-1.460577,-0.019746,0.03,2.276410e+10,0.00,0.23,27.626546,0.037316,0.0
26627,YTC,XNK Y tế TP.HCM,UPCoM,2021,0.044573,0.001241,0.02,0.000000e+00,0.01,0.28,27.383589,0.000000,0.0


In [49]:
df

,id,Company,Index,Year,ROE,ROA,CashRatio,ResearchDevFund,IAoverTA,TotalDebtoverTA,Firmsize_w,ResearchDevFund_ratio,SciTechFund_ratio
10,A32,Công ty 32,UPCoM,2016,0.172116,0.060481,0.47,5.585241e+10,0.00,0.22,26.865314,0.088759,0.0
11,A32,Công ty 32,UPCoM,2017,0.164296,0.057810,0.45,6.856909e+10,0.00,0.26,26.939857,0.112057,0.0
12,A32,Công ty 32,UPCoM,2018,0.189551,0.081188,0.22,8.093215e+10,0.00,0.01,26.873530,0.125240,0.0
13,A32,Công ty 32,UPCoM,2019,0.206177,0.106003,0.29,9.598561e+10,0.00,0.02,26.798452,0.138811,0.0
14,A32,Công ty 32,UPCoM,2020,0.174019,0.086321,0.18,1.124009e+11,0.00,0.00,26.914186,0.154273,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
26624,YTC,XNK Y tế TP.HCM,UPCoM,2018,0.640009,0.047042,0.08,2.049601e+10,0.00,0.45,27.368264,0.038055,0.0
26625,YTC,XNK Y tế TP.HCM,UPCoM,2019,0.554094,0.044125,0.21,2.276410e+10,0.00,0.37,27.346265,0.043871,0.0
26626,YTC,XNK Y tế TP.HCM,UPCoM,2020,-1.460577,-0.019746,0.03,2.276410e+10,0.00,0.23,27.626546,0.037316,0.0
26627,YTC,XNK Y tế TP.HCM,UPCoM,2021,0.044573,0.001241,0.02,0.000000e+00,0.01,0.28,27.383589,0.000000,0.0


In [56]:
df_clean=df[['ROE','CashRatio','IAoverTA','TotalDebtoverTA','Firmsize_w','ResearchDevFund_ratio']]

In [57]:
df_clean

,ROE,CashRatio,IAoverTA,TotalDebtoverTA,Firmsize_w,ResearchDevFund_ratio
10,0.172116,0.47,0.00,0.22,26.865314,0.088759
11,0.164296,0.45,0.00,0.26,26.939857,0.112057
12,0.189551,0.22,0.00,0.01,26.873530,0.125240
13,0.206177,0.29,0.00,0.02,26.798452,0.138811
14,0.174019,0.18,0.00,0.00,26.914186,0.154273
...,...,...,...,...,...,...
26624,0.640009,0.08,0.00,0.45,27.368264,0.038055
26625,0.554094,0.21,0.00,0.37,27.346265,0.043871
26626,-1.460577,0.03,0.00,0.23,27.626546,0.037316
26627,0.044573,0.02,0.01,0.28,27.383589,0.000000


In [58]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler

# Tạo DataFrame từ dữ liệu


# Scale dữ liệu
# Sử dụng StandardScaler
# regression = np.log(regression)
x=df_clean.drop('ROE',axis=1)
y=df_clean['ROE']
print(x.shape,y.shape)

# Check for infinite values
print("Infinite values in x:", np.any(np.isinf(x)))

# Check for missing values (NaNs)
print("Missing values (NaNs) in x:", np.any(np.isnan(x)))

# scaler = StandardScaler()
# Sử dụng MinMaxScaler
# scaler = MinMaxScaler()
# x = pd.DataFrame(x, columns=['TotalDebtoverTA'])  # Thay 'column_names' bằng tên cột thích hợp

# Thêm cột hằng 1 vào DataFrame để tính intercept trong mô hình
# x = np.column_stack((x, np.ones(x.shape[0])))
# x_values = x.values
# y_values = y.values

# Phân tích hồi quy tuyến tính
x=sm.add_constant(x)
model = sm.OLS(y, x)
results = model.fit()

# Tạo bảng thống kê tuyến tính
summary = results.summary()
print(summary)


(15654, 5) (15654,)
Infinite values in x: False
Missing values (NaNs) in x: False
                            OLS Regression Results                            
Dep. Variable:                    ROE   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                 -0.000
Method:                 Least Squares   F-statistic:                    0.9867
Date:                Fri, 10 May 2024   Prob (F-statistic):              0.424
Time:                        11:12:10   Log-Likelihood:                -39315.
No. Observations:               15654   AIC:                         7.864e+04
Df Residuals:                   15648   BIC:                         7.869e+04
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------

In [25]:
df

,id,Company,Index,Year,ROE,CashRatio,ResearchDevFund,IAoverTA,T_Debt1,Firmsize_w
10,A32,Công ty 32,UPCoM,2016,0.223213,0.47,5.585241e+10,0.00,4.731134e-13,26.865314
11,A32,Công ty 32,UPCoM,2017,0.201057,0.45,6.856909e+10,0.00,5.189700e-13,26.939857
12,A32,Công ty 32,UPCoM,2018,0.214608,0.22,8.093215e+10,0.00,2.132919e-14,26.873530
13,A32,Công ty 32,UPCoM,2019,0.210010,0.29,9.598561e+10,0.00,4.598438e-14,26.798452
14,A32,Công ty 32,UPCoM,2020,0.173840,0.18,1.124009e+11,0.00,0.000000e+00,26.914186
...,...,...,...,...,...,...,...,...,...,...
26624,YTC,XNK Y tế TP.HCM,UPCoM,2018,0.249297,0.08,2.049601e+10,0.00,5.852299e-13,27.368264
26625,YTC,XNK Y tế TP.HCM,UPCoM,2019,0.252424,0.21,2.276410e+10,0.00,4.918918e-13,27.346265
26626,YTC,XNK Y tế TP.HCM,UPCoM,2020,-0.892563,0.03,2.276410e+10,0.00,2.310316e-13,27.626546
26627,YTC,XNK Y tế TP.HCM,UPCoM,2021,0.490421,0.02,0.000000e+00,0.01,3.586050e-13,27.383589


In [27]:
df

,id,Company,Index,Year,ROE,CashRatio,ResearchDevFund,IAoverTA,T_Debt1,Firmsize_w
10,A32,Công ty 32,UPCoM,2016,0.223213,0.47,5.585241e+10,0.00,4.731134e-13,26.865314
11,A32,Công ty 32,UPCoM,2017,0.201057,0.45,6.856909e+10,0.00,5.189700e-13,26.939857
12,A32,Công ty 32,UPCoM,2018,0.214608,0.22,8.093215e+10,0.00,2.132919e-14,26.873530
13,A32,Công ty 32,UPCoM,2019,0.210010,0.29,9.598561e+10,0.00,4.598438e-14,26.798452
14,A32,Công ty 32,UPCoM,2020,0.173840,0.18,1.124009e+11,0.00,0.000000e+00,26.914186
...,...,...,...,...,...,...,...,...,...,...
26624,YTC,XNK Y tế TP.HCM,UPCoM,2018,0.249297,0.08,2.049601e+10,0.00,5.852299e-13,27.368264
26625,YTC,XNK Y tế TP.HCM,UPCoM,2019,0.252424,0.21,2.276410e+10,0.00,4.918918e-13,27.346265
26626,YTC,XNK Y tế TP.HCM,UPCoM,2020,-0.892563,0.03,2.276410e+10,0.00,2.310316e-13,27.626546
26627,YTC,XNK Y tế TP.HCM,UPCoM,2021,0.490421,0.02,0.000000e+00,0.01,3.586050e-13,27.383589


In [28]:
df.Year


10       2016
11       2017
12       2018
13       2019
14       2020
         ... 
26624    2018
26625    2019
26626    2020
26627    2021
26628    2022
Name: Year, Length: 19773, dtype: int64

In [26]:
import pandas as pd
import numpy as np
from regpyhdfe import Regpyhdfe

# from sklearn.datasets import load_boston

def sklearn_to_df(sklearn_dataset):
        df = pd.DataFrame(sklearn_dataset.data, columns=sklearn_dataset.feature_names)
        df['target'] = pd.Series(sklearn_dataset.target)
        return df

# Year=df.Year

# df = sklearn_to_df(load_boston())

# df.to_stata("boston.dta")
# . reghdfe target CRIM ZN INDUS NOX AGE, absorb(CHAS RAD)
# (MWFE estimator converged in 3 iterations)
#
# HDFE Linear regression                            Number of obs   =        506
# Absorbing 2 HDFE groups                           F(   5,    491) =      21.93
#                                                   Prob > F        =     0.0000
#                                                   R-squared       =     0.3887
#                                                   Adj R-squared   =     0.3712
#                                                   Within R-sq.    =     0.1825
#                                                   Root MSE        =     7.2929
#
# ------------------------------------------------------------------------------
#       target |      Coef.   Std. Err.      t    P>|t|     [95% Conf. Interval]
# -------------+----------------------------------------------------------------
#         CRIM |  -.2089114   .0491012    -4.25   0.000    -.3053857   -.1124371
#           ZN |   .0679261   .0183051     3.71   0.000       .03196    .1038922
#        INDUS |  -.2279553   .0860909    -2.65   0.008    -.3971074   -.0588033
#          NOX |  -9.424849   5.556005    -1.70   0.090    -20.34133     1.49163
#          AGE |  -.0140739   .0183467    -0.77   0.443    -.0501215    .0219738
#        _cons |   31.24755    2.53596    12.32   0.000     26.26487    36.23022
# ------------------------------------------------------------------------------
#
# Absorbed degrees of freedom:
# -----------------------------------------------------+
#  Absorbed FE | Categories  - Redundant  = Num. Coefs |
# -------------+---------------------------------------|
#         CHAS |         2           0           2     |
#          RAD |         9           1           8     |
# -----------------------------------------------------+

# target~CRIM + ZN + INDUS + NOX + AGE, absorb(CHAS, RAD)
#                                  OLS Regression Results
# =======================================================================================
# Dep. Variable:                 target   R-squared (uncentered):                   0.183
# Model:                            OLS   Adj. R-squared (uncentered):              0.158
# Method:                 Least Squares   F-statistic:                              21.93
# Date:                Mon, 11 Jan 2021   Prob (F-statistic):                    7.57e-20
# Time:                        20:30:53   Log-Likelihood:                         -1715.7
# No. Observations:                 506   AIC:                                      3441.
# Df Residuals:                     491   BIC:                                      3463.
# Df Model:                           5
# Covariance Type:            nonrobust
# ==============================================================================
#                  coef    std err          t      P>|t|      [0.025      0.975]
# ------------------------------------------------------------------------------
# CRIM          -0.2089      0.049     -4.255      0.000      -0.305      -0.112
# ZN             0.0679      0.018      3.711      0.000       0.032       0.104
# INDUS         -0.2280      0.086     -2.648      0.008      -0.397      -0.059
# NOX           -9.4248      5.556     -1.696      0.090     -20.341       1.492
# AGE           -0.0141      0.018     -0.767      0.443      -0.050       0.022
# ==============================================================================
# Omnibus:                      172.457   Durbin-Watson:                   0.904
# Prob(Omnibus):                  0.000   Jarque-Bera (JB):              532.297
# Skew:                           1.621   Prob(JB):                    2.59e-116
# Kurtosis:                       6.839   Cond. No.                         480.
# ==============================================================================
#
# Notes:
# [1] R² is computed without centering (uncentered) since the model does not contain a constant.
# [2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
model = Regpyhdfe(df_clean, 'ROA', ['CashRatio','ResearchDevFund','IAoverTA','TotalDebtoverTA','Firmsize_w','ResearchDevFund_ratio','SciTechFund_ratio'])
results = model.fit()
# print("target~CRIM + ZN + INDUS + NOX 
# + AGE, absorb(CHAS, RAD)")
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                    ROA   R-squared:                       0.012
Model:                            OLS   Adj. R-squared:                  0.011
Method:                 Least Squares   F-statistic:                     31.33
Date:                Fri, 10 May 2024   Prob (F-statistic):           1.15e-37
Time:                        11:07:33   Log-Likelihood:                -1937.4
No. Observations:               15654   AIC:                             3889.
Df Residuals:                   15647   BIC:                             3942.
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
CashRatio                 0.00